# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [1]:
!pip install -qU ragas==0.2.10

In [2]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0 langchain_community

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [3]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

Please enter your OpenAI API key!··········


OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [4]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

Please enter your Ragas API key!··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [5]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [6]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31427    0 31427    0     0  72382      0 --:--:-- --:--:-- --:--:-- 72579


In [7]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70286    0 70286    0     0   513k      0 --:--:-- --:--:-- --:--:--  516k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
import langchain_community
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht are the new capabilitees introdused by Gem...,[Voice and live camera mode are science fictio...,Gemini 1.5 Pro introduced several brand new ca...,single_hop_specifc_query_synthesizer
1,Wht is OpenAI's role in the advancemnt of mult...,[the then-new GPT-4 Turbo and $1/mTok for GPT-...,OpenAI has contributed to the advancement of m...,single_hop_specifc_query_synthesizer
2,What recent feature has Google Gemini introduc...,[ChatGPT voice mode now provides the option to...,Google Gemini has introduced a feature that al...,single_hop_specifc_query_synthesizer
3,What advancements has Apple made in machine le...,[I’m beginning to see the most popular idea of...,"Apple's MLX library, an array framework for Ap...",single_hop_specifc_query_synthesizer
4,How has prompt-driven app generation become a ...,[<1-hop>\n\nVoice and live camera mode are sci...,Prompt-driven app generation has become a comm...,multi_hop_abstract_query_synthesizer
5,How has the concept of universal access to AI ...,[<1-hop>\n\nVoice and live camera mode are sci...,The concept of universal access to AI models e...,multi_hop_abstract_query_synthesizer
6,What are the current challenges and developmen...,[<1-hop>\n\nVoice and live camera mode are sci...,The current challenges in AI agents primarily ...,multi_hop_abstract_query_synthesizer
7,How has the development of prompt-driven app g...,[<1-hop>\n\nVoice and live camera mode are sci...,"In 2024, prompt-driven app generation has beco...",multi_hop_abstract_query_synthesizer
8,How has the development of Llama 3 and Llama 2...,"[<1-hop>\n\nhere, but getting to that value is...",The development of Llama 3 and Llama 2 has sig...,multi_hop_specific_query_synthesizer
9,How has the development of Claude 3 and its it...,[<1-hop>\n\nit easier for a model to follow th...,The development of Claude 3 and its iterations...,multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [12]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/c0c9a03d-9726-49f5-b480-0e99b7f0e9e8


'https://app.ragas.io/dashboard/alignment/testset/c0c9a03d-9726-49f5-b480-0e99b7f0e9e8'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [13]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

74

#### ❓ Question:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

https://github.com/rprav007/aie5_hw8/tree/main

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [15]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [16]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [17]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [18]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [19]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [20]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [22]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [23]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [24]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [25]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [26]:
response["response"]

"LLM agents are considered useful for several reasons:\n\n1. **Ease of Construction**: They can be built with relatively simple code—often just a few hundred lines of Python—provided there is sufficient training data. This makes them more accessible than previously thought, no longer being exclusive to super-rich entities.\n\n2. **Local Deployment**: Recent advancements have allowed LLMs to be run on personal devices, making them more practical for individual users. For example, Meta's release of Llama and subsequent developments have enabled LLMs to operate on standard computers.\n\n3. **Effective Code Generation**: LLMs have shown particular strength in generating code. They can produce functional code snippets and even self-correct through tools like the ChatGPT Code Interpreter, which allows them to test and refine code iteratively.\n\n4. **Potential for AI Agents**: There is excitement around the idea of AI agents that can act on behalf of users. While still in the prototype stage

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [27]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [28]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wht are the new capabilitees introdused by Gem...,[I wrote about this at the time in The killer ...,[Voice and live camera mode are science fictio...,The new capabilities introduced by Gemini 1.5 ...,Gemini 1.5 Pro introduced several brand new ca...,single_hop_specifc_query_synthesizer
1,Wht is OpenAI's role in the advancemnt of mult...,"[In 2024, almost every significant model vendo...",[the then-new GPT-4 Turbo and $1/mTok for GPT-...,OpenAI's role in the advancement of multi-moda...,OpenAI has contributed to the advancement of m...,single_hop_specifc_query_synthesizer
2,What recent feature has Google Gemini introduc...,[OpenAI aren’t the only group with a multi-mod...,[ChatGPT voice mode now provides the option to...,Google Gemini has recently introduced a featur...,Google Gemini has introduced a feature that al...,single_hop_specifc_query_synthesizer
3,What advancements has Apple made in machine le...,"[Apple Intelligence is bad, Apple’s MLX librar...",[I’m beginning to see the most popular idea of...,Apple has made significant advancements in mac...,"Apple's MLX library, an array framework for Ap...",single_hop_specifc_query_synthesizer
4,How has prompt-driven app generation become a ...,[Anthropic kicked this idea into high gear whe...,[<1-hop>\n\nVoice and live camera mode are sci...,Prompt-driven app generation has become a comm...,Prompt-driven app generation has become a comm...,multi_hop_abstract_query_synthesizer
5,How has the concept of universal access to AI ...,[I wrote about this at the time in The killer ...,[<1-hop>\n\nVoice and live camera mode are sci...,The concept of universal access to AI models h...,The concept of universal access to AI models e...,multi_hop_abstract_query_synthesizer
6,What are the current challenges and developmen...,[A lot of people are excited about AI agents—a...,[<1-hop>\n\nVoice and live camera mode are sci...,The current challenges in AI agents include th...,The current challenges in AI agents primarily ...,multi_hop_abstract_query_synthesizer
7,How has the development of prompt-driven app g...,[These abilities are just a few weeks old at t...,[<1-hop>\n\nVoice and live camera mode are sci...,"In 2024, prompt-driven app generation has sign...","In 2024, prompt-driven app generation has beco...",multi_hop_abstract_query_synthesizer
8,How has the development of Llama 3 and Llama 2...,[Code may be the best application\n\nThe ethic...,"[<1-hop>\n\nhere, but getting to that value is...",The development of Llama 3 and Llama 2 has sig...,The development of Llama 3 and Llama 2 has sig...,multi_hop_specific_query_synthesizer
9,How has the development of Claude 3 and its it...,[Getting back to models that beat GPT-4: Anthr...,[<1-hop>\n\nit easier for a model to follow th...,The development of Claude 3 and its iterations...,The development of Claude 3 and its iterations...,multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [29]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [30]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [31]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[22]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-KEkyoe4K9WHAtKUPJrUOZCjX on tokens per min (TPM): Limit 30000, Used 29867, Requested 1857. Please try again in 3.448s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[7]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-KEkyoe4K9WHAtKUPJrUOZCjX on tokens per min (TPM): Limit 30000, Used 29144, Requested 2206. Please try again in 2.7s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[13]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-KEkyoe4K9WHAtKUPJrUOZCj

{'context_recall': 0.8708, 'faithfulness': nan, 'factual_correctness': 0.4791, 'answer_relevancy': 0.9524, 'context_entity_recall': 0.4238, 'noise_sensitivity_relevant': 0.3964}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [32]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

Please enter your Cohere API key!··········


In [33]:
!pip install -qU cohere langchain_cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.9 MB/s eta 0:00:00



We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [34]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [35]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [36]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [37]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

'LLM agents can be useful primarily in writing code, as they have shown a strong capability in this area. The grammar rules of programming languages are less complex than those of natural languages, which contributes to their effectiveness in generating code. However, there is skepticism surrounding their overall utility due to issues like gullibility—where LLMs may accept false information as true—making them unreliable for tasks that require making meaningful decisions or discerning truth from fiction. Additionally, there is a call for better criticism and discussion regarding the ethical implications and potential negative impacts of using LLMs, emphasizing the need for responsible usage that balances positive and negative outcomes.'

In [38]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [39]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[13]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-KEkyoe4K9WHAtKUPJrUOZCjX on tokens per min (TPM): Limit 30000, Used 29040, Requested 2152. Please try again in 2.384s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[4]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-KEkyoe4K9WHAtKUPJrUOZCjX on tokens per min (TPM): Limit 30000, Used 29354, Requested 1731. Please try again in 2.17s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[25]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-KEkyoe4K9WHAtKUPJrUOZC

{'context_recall': 0.8333, 'faithfulness': 0.5714, 'factual_correctness': 0.5255, 'answer_relevancy': 0.9510, 'context_entity_recall': 0.4409, 'noise_sensitivity_relevant': 0.2436}

#### ❓ Question:

Which system performed better, on what metrics, and why?

https://github.com/rprav007/aie5_hw8/tree/main